In [65]:
import pandas as pd
resumes = pd.read_csv("UpdatedResumeDataSet.csv", header=None, skiprows = 1)

resumes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 962 entries, 0 to 961
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       962 non-null    object
 1   1       962 non-null    object
dtypes: object(2)
memory usage: 15.2+ KB


In [66]:
print("Number of nulls:", resumes.isnull().values.sum())

Number of nulls: 0


In [67]:
#Deal with the erroneous values in the dataset

resumes[1] = resumes[1].replace('[^a-zA-Z0-9 ]', ' ', regex=True)
resumes[1] = resumes[1].replace("â¢", '')


In [68]:
# resumes[1]
resumes.to_csv("resume_clean.csv", index=False)

In [69]:
#Association rule mining of resume

In [70]:
skills = [['node.js', 'react.js', 'Javascript'],
           ['Javascript', 'Python', 'Java'],
           ['C++', 'Java', 'Python', 'C'],
           ['react.js', 'node.js', 'Python']]
print(skills)

[['node.js', 'react.js', 'Javascript'], ['Javascript', 'Python', 'Java'], ['C++', 'Java', 'Python', 'C'], ['react.js', 'node.js', 'Python']]


In [71]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
skills_array = te.fit(skills).transform(skills)
skills_array

array([[False, False, False,  True, False,  True,  True],
       [False, False,  True,  True,  True, False, False],
       [ True,  True,  True, False,  True, False, False],
       [False, False, False, False,  True,  True,  True]])

In [72]:
import pandas as pd
skills_df = pd.DataFrame(skills_array, columns = te.columns_)
skills_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   C           4 non-null      bool 
 1   C++         4 non-null      bool 
 2   Java        4 non-null      bool 
 3   Javascript  4 non-null      bool 
 4   Python      4 non-null      bool 
 5   node.js     4 non-null      bool 
 6   react.js    4 non-null      bool 
dtypes: bool(7)
memory usage: 156.0 bytes


In [73]:
from mlxtend.frequent_patterns import apriori, association_rules
frequent_itemsets = apriori(skills_df, min_support=0.4, use_colnames=True)
#frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
#frequent_itemsets[frequent_itemsets['length'] == 2].count()
frequent_itemsets

,support,itemsets
0,0.50,(Java)
1,0.50,(Javascript)
2,0.75,(Python)
3,0.50,(node.js)
4,0.50,(react.js)
5,0.50,"(Java, Python)"
6,0.50,"(react.js, node.js)"


In [74]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.9)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Java),(Python),0.5,0.75,0.5,1.0,1.333333,0.125,inf
1,(react.js),(node.js),0.5,0.50,0.5,1.0,2.000000,0.250,inf
2,(node.js),(react.js),0.5,0.50,0.5,1.0,2.000000,0.250,inf


In [91]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
newcols = pd.DataFrame(ohe.fit_transform(skills_df).toarray())
newcols.columns = ["Skills" + str(x) for x in newcols.columns]
ohe_col1 = newcols
ohe_col1
newcols

,Skills0,Skills1,Skills2,Skills3,Skills4,Skills5,Skills6,Skills7,Skills8,Skills9,Skills10,Skills11,Skills12,Skills13
0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
1,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
2,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
3,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
